### IPMOF - InterPenetrating Metal Organic Frameworks

#### Initialize necessary information

1. Read FF_Parameters excel file to get force field parameters for atoms
2. Initialize force field selection, cut_off radius and grid size for energy map

#### Read structural information for MOF files in a given directory
1. Read MOF files in ".mol2" format from given directory and create a list
2. Create MOF objects for structure files
3. Initialize structural information for the MOFs

#### Read simulation input parameters
1. Read simulation parameters from input file

#### Calculate energy map
1. Determine packing amount of the MOF
2. Calculate packed coordinates for the base MOF
3. Calculate energy map

#### Start interpenetration
1. Energy map + mobile_mof

In [16]:
import os
import math
import time

# Load 3rd party libraries
import yaml

# Load interpenetration python libraries
from ipmof.crystal import MOF
from ipmof.forcefield import read_ff_parameters
from ipmof.energymap import energy_map, get_mof_list, get_uniq_atom_list
from ipmof.interpenetration import run_interpenetration, check_extension, save_extension
from ipmof.core import core_mof_properties, core_mof_sort, core_mof_dir
# --------------------------------------------------------------------------------------------------
from ipmof.parameters import sim_dir_data as sim_dir
from ipmof.parameters import sim_par_data as sim_par

start_time = time.clock()

# Read excel file containing force field information
force_field = read_ff_parameters(sim_dir['force_field_path'], sim_par['force_field'])

### File Input Options
- <b> Built-in read_mol2 </b>
<pre>
uc_size, uc_angle, atom_names, atom_coors = read_mol2(mol2_path)
mof = MOF()
mof.initialize(mof.mol2_path)
</pre> 
- <b> Ase pdb read (cif read gives error) </b>
<pre>
from ase.io import read
mof_atoms = read(mof_dir, format='pdb')
mof_atoms.get_positions()                # Coordinates
mof_atoms.get_chemical_symbols()         # Atom names
mof_atoms.get_cell()                     # Cell vectors? (check)
mof_obj.get_number_of_atoms()            # Num of atoms
mof_obj.get_volume()                     # Unit cell volume

</pre>
- <b> Open babel </b>
<pre>
babel -icif *.cif -opdb *.pdb
</pre>

In [17]:
# Create list of MOFs
# mol2_list = get_mof_list(sim_dir['mol2_dir'], '.mol2')

# Create MOf list from CoRE database
mof_properties = core_mof_properties(sim_dir['core_path'])

sorted_mofs = core_mof_sort(mof_properties, sort='void_fraction', limit=0.85)
mol2_list = core_mof_dir(sorted_mofs, sim_dir['mol2_dir'])

# Select MOF couple
base_mof_index = 4
mobile_mof_index = 4

Gathered a total of 96 MOFs
With void_fraction > 0.85
91 mofs are missing
5 total mofs found


In [18]:
# Read mol2 files and initialize MOF objects
mol2_path = os.path.join(sim_dir['mol2_dir'], mol2_list[base_mof_index])
base_mof = MOF(mol2_path)
base_mof.force_field(force_field)
print('Base MOF selected as: ', base_mof.name)

mol2_path = os.path.join(sim_dir['mol2_dir'], mol2_list[mobile_mof_index])
mobile_mof = MOF(mol2_path)
mobile_mof.force_field(force_field)
print('Mobile MOF selected as: ', mobile_mof.name)

Base MOF selected as:  XEBHOC_clean
Mobile MOF selected as:  XEBHOC_clean


In [19]:
extended_structure = base_mof.extend_unit_cell(sim_par['cut_off'])

print('Base MOF unit cell: ', base_mof.uc_size)
print('Packing factor:', base_mof.packing_factor)
print('Num of coor :', len(base_mof.packed_coors)*len(base_mof.packed_coors[0]))

Base MOF unit cell:  [15.376, 19.8482, 19.8482]
Packing factor: [3, 3, 3]
Num of coor : 3564


In [20]:
init_time = time.clock() - start_time

atom_list = get_uniq_atom_list([mobile_mof])
print('Calculating emap for', base_mof.name, 'with atoms:', atom_list['atom'])
emap = energy_map(sim_par, base_mof, atom_list)

emap_time = time.clock() - start_time

Calculating emap for XEBHOC_clean with atoms: ['H', 'Cu', 'N', 'O', 'C']


In [21]:
# Run interpenetration
summary, new_structures = run_interpenetration(sim_par, base_mof, mobile_mof, emap, atom_list)

interpenetration_time = time.clock() - start_time

In [22]:
# Get minimum energy structure by sorting total structure energies
min_energy_structure = sorted(new_structures, key=lambda k: k['energy'])[0]

In [23]:
# Check for collision in the extended unitcell of new structure and energy map
collision = check_extension(sim_par, base_mof, mobile_mof, emap, atom_list, min_energy_structure)

extension_time = time.clock() - start_time
print('Collision:', collision)

Collision: True


In [24]:
# Get structure information for the interpenetrating structure
ext_structure = save_extension(sim_par, base_mof, mobile_mof, emap, atom_list, min_energy_structure)

In [25]:
# Extend MOF coordinates and get atom names and coordinates of extended unit cells of MOF object
extended_structure = base_mof.extend_unit_cell(sim_par['cut_off'])

# Create new MOF objects for base and mobile MOFs
ext_base_mof = MOF(extended_structure, file_format='dict')
ext_mobile_mof = MOF(ext_structure, file_format='dict')

# Join base and mobile structure layers
joined_mof = ext_base_mof.join(ext_mobile_mof, colorify=sim_par['export_colorify'])

# Export to xyz format
joined_mof.export(sim_dir['export_dir'], file_format=sim_par['export_format'])

export_time = time.clock() - start_time